In [1]:
%matplotlib inline
import geopandas
import pandas
import os
import glob

import dateutil
import datetime

from simpledbf import Dbf5
import pandas_access as mdb


pandas.set_option("display.max_columns",999)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
tax_sales = pandas.DataFrame.from_csv('/home/clay/Downloads/Land Tax Sale 186_ St Louis, MO - Land-Tax-Sale-186.csv',index_col=None)
tax_sales.loc[:,'Address'] = tax_sales.Address.apply(lambda x: x[:-1] if x[-1]==' ' else x)
tax_sales.loc[:,'Total'] = tax_sales.Total.apply(lambda x: int(x.replace(',','').replace('.00','').replace('$','')))
#pandas.merge(parcels,tax_sales,left_on='SITEADDR',right_on='Address').to_csv('tax_joined_to_parcels.csv')
tax_sales.head()

,Land Tax #,Owner,Address,Total
0,186-006,"Williams, Contrell",3429 N 14TH ST,1047
1,186-007,Askins Development Group Llc,2737 UTAH ST,1486
2,186-008,"Atoyebi, Adebola Esther",3839 MICHIGAN AV,1963
3,186-009,"Baker, George & Lucy",3849 S COMPTON AV,1443
4,186-010,Bank Of New York,1904 COLEMAN AV,1509


In [3]:
# Parcel Shape file
prcl_shp = geopandas.GeoDataFrame.from_file('workdir/downloads/prcl.shp')

# Join Parcel Info
Prcl =  mdb.read_table("workdir/downloads/prcl.mdb", "Prcl")
parcels = pandas.merge(prcl_shp,Prcl,left_on='HANDLE',right_on='Handle')
parcels.loc[:,'Nbrhd']= parcels.Nbrhd.astype(int)

# Join in Neighborhood names
neighborhoods = geopandas.read_file('workdir/downloads/BND_Nhd88_cw.shp')
parcels = pandas.merge(parcels,neighborhoods[['NHD_NAME','NHD_NUM']],left_on='Nbrhd',right_on='NHD_NUM')

# Join More Parcel info, this one has nice address names to join to CSB data
par = Dbf5('workdir/downloads/par.dbf',codec='cp1250').to_dataframe()
parcels = pandas.merge(parcels,par,on='HANDLE')

del prcl_shp
del Prcl
del par

parcels.head()

,HANDLE,geometry,CityBlock,Parcel,OwnerCode,ParcelId,PrimAddrRecNum,AddrType,LowAddrNum,LowAddrSuf,HighAddrNum,HighAddrSuf,NLC,Parity,StPreDir,StName,StType,StSufDir,StdUnitNum,OwnerName,OwnerName2,OwnerAddr,OwnerCity,OwnerState,OwnerCountry,OwnerZIP,LegalDesc1,LegalDesc2,LegalDesc3,LegalDesc4,LegalDesc5,AsrClassCode,AsrLandUse1,AsrLanduse2,RedevPhase,RedevYearEnd,RedevPhase2,RedevYearEnd2,VacantLot,SpecBusDist,SpecBusDist2,TIFDist,LendingAgcy,Condominium,NbrOfUnits,NbrOfApts,Frontage,LandArea,RecDailyDate,RecDailyNum,RecBookNum,RecPageNum,AsdLand,AsdImprove,AsdTotal,BillLand,BillImprove,BillTotal,AprLand,CostAprImprove,AsmtAppealYear,AsmtAppealNum,AsmtAppealType,PriorAsdDate,PriorAsdLand,PriorAsdImprove,PriorAsdTotal,PriorTaxAmt,CDALandUse1,CDALandUse2,LRMSUnitNum,Zoning,NbrOfBldgsRes,NbrOfBldgsCom,FirstYearBuilt,LastYearBuilt,ResSalePrice,ResSaleDate,VacBldgYear,GeoCityBlockPart,Ward10,Precinct10,InspArea10,Ward00,Precinct02,Precinct04,Nbrhd,CDADist,CDASubDist,PoliceDist,CensTract10,CensBlock10,CensBlock00,Ward90,Precinct90,CensBlock90,HouseConsDist,AsrNbrhd,EntZone,ImpactArea,CTDArea,LeafArea,ZIP_x,OnFloodBlock,SpecParcelType,SubParcelType,NbrOfSubAccts,NbrOfCondos,LRMSParcel,AcctPrimary,GisPrimary,GisCityBLock,GisParcel,GisOwnerCode,Handle,Parcel9,OwnerOcc,FirstDate,LastDate,OwnerUpdate,NHD_NAME,NHD_NUM,UPDATED,SITEADDR,OWNERNAME,OWNERNAME2,OWNERADDR,OWNERCITY,OWNERSTATE,OWNERCNTRY,OWNERZIP,OWNERGROUP,NUMUNITS,ZONING1,ZONING2,ZONING3,VACANTLAND,ASMTLAND,ASMTIMPROV,ASMTTOTAL,LANDUSE1,LANDUSE2,LANDUSE3,LANDUSE4,ASRUSE1,ASRUSE2,ASRUSE3,ASRUSE4,ASRCLASS1,ASRCLASS2,ASRCLASS3,ASRCLASS4,LEGAL1,LEGAL2,LEGAL3,LEGAL4,LEGAL5,NUMBLDGS,LANDAREA,FRONTAGE,DAILYDATE,DAILYNUM,BOOKNUM,BOOKPAGE,BDG1YEAR,BDG1AREA,BDG1EXWALL,BDG1STRYCD,BDG1OCCCD,CITYBLOCK,PARCEL,PARITY,ADDRNUM,ADDRSUF,STREETPRE,STREETNAME,STREETSUF,STRSUFDIR,UNITNUM,CDADIST,CDASUBDIST,WARD,PRECINCT02,PRECINCT04,NBRHD,ASRNBRHD,CENSBLOCK,POLICEDIST,ZIP_y,IMPACTAREA,HSCONSERV,OWNEROCC,VACBLDGYR,PARCEL10,CENTRACT10,CENBLOCK10,WARD10,PRECINCT10,INSPAREA10,LEAFAREA,BLOCKPART
0,10001000005,"POLYGON ((589498.305113 1014298.393394, 589476...",1.0,5,0,00010000050,1,3,626,NaN,626,NaN,785.0,E,S,1ST,ST,NaN,NaN,CITY OF ST LOUIS,NaN,1200 MARKET ST,ST LOUIS MO,NaN,NaN,63103,CB 1 FIRST ST,0.140 ACS,BROWNS SURVEY ADDN LOT PTS 9 THRU 12,BD N-POPCAR S- PLUM,NaN,4,3000,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000000e+00,6098,NaN,0,CVN,0,6800.0000,-2300.0000,4400.0000,6800.0000,-2300.0000,4400.0000,21300.0000,0.0000,0,0,NaN,10/04/15 00:00:00,6800.0000,-2300.0000,4400.0000,0.0000,6700,0,NaN,NaN,0,0,0,0,0.0000,NaN,0,1.0085,7,6,3,7,12,6,35,9,2,4,1274.0,1054,1222.1001,7,6,1222.104,55,266,1,0,0,0,63102.0,1,NaN,NaN,1,0,1,1,1,1.0,5,0,10001000005,100005,NaN,01/25/09 00:00:00,04/28/17 00:00:00,05/10/83 00:00:00,Downtown,35,2017-04-28,626 S 1ST ST,CITY OF ST LOUIS,NaN,1200 MARKET ST,ST LOUIS MO,NaN,NaN,63103,SL,0,NaN,NaN,NaN,Y,6800,-2300,4400,6700,5000,0,0,3000,5000,0,0,4,200,0,0,CB 1 FIRST ST,0.140 ACS,BROWNS SURVEY ADDN LOT PTS 9 THRU 12,BD N-POPCAR S- PLUM,NaN,0,6098,0.0,NaN,0,CVN,0,0,0,0,0,0,1.0,5,E,626,NaN,S,1ST,ST,NaN,NaN,9,2,7,12,6,35,266,1222.1001,4,63102,0,55,NaN,0,1000005,1274.0,1054,7,6,3,0,1.0085
1,10001000005,"POLYGON ((589498.305113 1014298.393394, 589476...",1.0,5,8,00010000058,1,3,626,NaN,626,NaN,785.0,E,S,1ST,ST,NaN,NaN,CITY OF ST LOUIS,C/O DDI MEDIA,8315 DRURY IND PARKWAY,ST LOUIS,MO,NaN,63114,C.B. 0001 S 1ST ST,BB # 254,DDI-MEDIA,NaN,NaN,200,5000,0,0,0,0,0,0,0,0,0,210,0,0,0,9.99999978e-03,0,NaN,0,NaN,0,0.0000,12100.0000,12100.0000,0.0000,12100.0000,12100.0000,0.0000,17900.0000,0,0,NaN,10/04/15 00:00:00,0.0000,12100.0000,12100.0000,1044.2700,5000,0,NaN,NaN,0,0,0,0,0.0000,NaN,0,1.0085,7,6,3,7,12,6,35,9,2,4,1274.0,1054,1222.1001,7,6,1222.104,55,266,1,0,0,0,63102.0,1,NaN,E,0,0,1,0,0,1.0,5,0,10001000005,100005,NaN,07/23/06 00:00:00,04/28/17 00:00:00,07/20/06 00:00:00,Downtown,35,2017-04-28,626 S 1ST ST,CITY OF ST LOUIS,NaN,1200 MARKET ST,ST LOUIS MO,NaN,NaN,63103,SL,0,NaN,NaN,NaN,Y,6800,-2300,4400,6700,50

In [4]:
parcels = pandas.merge(parcels,tax_sales,left_on='SITEADDR',right_on='Address',how='left')

parcels.loc[:,'geometry_centroid'] = parcels.centroid

In [10]:
sum(~parcels['Address'].isnull())

1039

In [11]:
parcels[~parcels['Address'].isnull()].to_csv('parcels_out.csv')